### Installation

# Finetune Llama-3 with LLaMA Factory

Project homepage: https://github.com/hiyouga/LLaMA-Factory

In [6]:
!echo Y |pip uninstall backports.tarfile
!echo Y |pip install backports.tarfile

Found existing installation: backports.tarfile 1.2.0
Uninstalling backports.tarfile-1.2.0:
  Would remove:
    /opt/conda/lib/python3.10/site-packages/backports.tarfile-1.2.0.dist-info/*
    /opt/conda/lib/python3.10/site-packages/backports/*
  Would not remove (might be manually added):
    /opt/conda/lib/python3.10/site-packages/backports/functools_lru_cache.py
Proceed (Y/n)?   Successfully uninstalled backports.tarfile-1.2.0
  Using cached backports.tarfile-1.2.0-py3-none-any.whl.metadata (2.0 kB)
Using cached backports.tarfile-1.2.0-py3-none-any.whl (30 kB)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [7]:
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install -e .[torch,bitsandbytes]

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 456, done.
remote: Counting objects: 100% (456/456), done.
remote: Compressing objects: 100% (361/361), done.
remote: Total 456 (delta 122), reused 296 (delta 77), pack-reused 0 (from 0)
Receiving objects: 100% (456/456), 5.16 MiB | 4.14 MiB/s, done.
Resolving deltas: 100% (122/122), done.
/home/jovyan/LLaMA-Factory
assets/       docker/    Makefile        README.md         scripts/  tests/
CITATION.cff  examples/  MANIFEST.in     README_zh.md      setup.py  tests_v1/
data/         LICENSE    pyproject.toml  requirements.txt  src/
Obtaining file:///home/jovyan/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while

### Check GPU environment

In [8]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

#### Update Identity Dataset

In [9]:
import json

%cd /home/jovyan/LLaMA-Factory

NAME = "Llama-3"
AUTHOR = "LLaMA Factory"

with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

for sample in dataset:
  sample["output"] = sample["output"].replace("{{"+ "name" + "}}", NAME).replace("{{"+ "author" + "}}", AUTHOR)

with open("data/identity.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)

/home/jovyan/LLaMA-Factory


### Fine-tune model via Command Line

In [ ]:
import json

args = dict(
  stage="sft",                                               # do supervised fine-tuning
  do_train=True,
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  dataset="identity,alpaca_en_demo",                         # use alpaca and identity datasets
  template="llama3",                                         # use llama3 prompt template
  finetuning_type="lora",                                    # use LoRA adapters to save memory
  lora_target="all",                                         # attach LoRA adapters to all linear layers
  output_dir="llama3_lora",                                  # the path to save LoRA adapters
  per_device_train_batch_size=2,                             # the micro batch size
  gradient_accumulation_steps=4,                             # the gradient accumulation steps
  lr_scheduler_type="cosine",                                # use cosine learning rate scheduler
  logging_steps=5,                                           # log every 5 steps
  warmup_ratio=0.1,                                          # use warmup scheduler
  save_steps=1000,                                           # save checkpoint every 1000 steps
  learning_rate=5e-5,                                        # the learning rate
  num_train_epochs=3.0,                                      # the epochs of training
  max_samples=500,                                           # use 500 examples in each dataset
  max_grad_norm=1.0,                                         # clip gradient norm to 1.0
  loraplus_lr_ratio=16.0,                                    # use LoRA+ algorithm with lambda=16.0
  fp16=True,                                                 # use float16 mixed precision training
  report_to="none",                                          # disable wandb logging
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /home/jovyan/LLaMA-Factory

!llamafactory-cli train train_llama3.json

/home/jovyan/LLaMA-Factory
/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
[INFO|2025-12-03 09:13:14] llamafactory.hparams.parser:468 >> Process rank: 0, world size: 1, device: cuda:0, distributed training: False, compute dtype: torch.float16
tokenizer_config.json: 51.1kB [00:00, 52.0MB/s]
tokenizer.json: 9.09MB [00:00, 140MB/s]
special_tokens_map.json: 100%|█████████████████| 345/345 [00:00<00:00, 5.32MB/s]
[INFO|tokenization_utils_base.py:2095] 2025-12-03 09:13:16,979 >> loading file tokenizer.json from cache at /home/jovyan/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/fd5a4dc328319c1cfe9489eccfb9c6406bdfd469/tokenizer.json
[INFO|tokenization_utils_base.py:2095] 2025-12-03 09:13:16,979 >> loading file tokenizer.model from cache at None
[INFO|tokenizat

### Infer the fine-tuned model 
Type "exit" to end

In [ ]:
%cd /home/jovyan/LLaMA-Factory/src
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc



args = dict(
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="/home/jovyan/LLaMA-Factory/llama3_lora",                        # load the saved LoRA adapters
  template="llama3",                                         # same to the one in training
  finetuning_type="lora",                                    # same to the one in training
)
chat_model = ChatModel(args)

messages = []
print("Welcome to the CLI application, use `clear` to remove the history, use `exit` to exit the application.")
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("History has been removed.")
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)

  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

torch_gc()

### Merge the LoRA adapter and optionally upload model

In [ ]:
!huggingface-cli login

In [ ]:
import json

args = dict(
  model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct", # use official non-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="llama3_lora",                       # load the saved LoRA adapters
  template="llama3",                                        # same to the one in training
  finetuning_type="lora",                                   # same to the one in training
  export_dir="llama3_lora_merged",                          # the path to save the merged model
  export_size=2,                                            # the file shard size (in GB) of the merged model
  export_device="cpu",                                      # the device used in export, can be chosen from `cpu` and `auto`
  # export_hub_model_id="your_id/your_model",               # the Hugging Face hub ID to upload model
)

json.dump(args, open("merge_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /home/jovyan/LLaMA-Factory/LLaMA-Factory

!llamafactory-cli export merge_llama3.json